# 2 Naive Bayes Classifier

Assume that all classes are equally likely, i.e. the priors are $p(y=k)=1/C$ with $C$ the number of classes.  
The decision rule is defined by  
$$
\hat{y}=\argmax_k\Bigg(\prod_{j=1}p_j(x_j|y=k)\Bigg)
$$
where $p_j(x_j|y=k)$ are 1-dimensional histograms for each feature $j$ and class $k$.  
Rewrite to
$$
\hat{y}=\argmax_k\Bigg(\sum_{j=1}\log p_j(x_j|y=k)\Bigg)
$$
since tiny numbers are prone to numerical inaccuracy.

> Implement training of the naive Bayes classifier as a function  
> `histograms, binning = fit_naive_bayes(features, labels, bincount)`  
> where `histograms` is the $C×D×L$ array if histograms ($D$ is the number of feature dimensions, $L$ the number of bins), and `binning` is a $C×D×2$ array describing the bin layout.

In [7]:
from typing import Tuple
import numpy as np
from sklearn import model_selection

In [8]:
from sklearn.datasets import load_digits


digits = load_digits()

print(digits.keys())

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]


dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])


In [9]:
def IQR(features) -> float:
    """Calculates the IQR of the given features."""
    q75, q25 = np.percentile(
        features, [0.75, 0.25], axis=0, interpolation="nearest")
    return q75 - q25


def freedman_diaconis(features, labels) -> np.ndarray:
    """Returns the bins."""
    klasses = np.unique(labels)
    bins = np.zeros((klasses.size, features.shape[1]))
    binwidths = np.zeros((klasses.size, features.shape[1]))
    for k, klass in enumerate(klasses):
        klass_features = features[labels == klass]
        h = (2*IQR(klass_features)/np.cbrt(klass_features.shape[0]))
       
        h[h==0] = np.inf  # cant divide by 0

        binwidths[k] = h
        bins[k] = np.ceil(
            (np.max(klass_features, axis=0)-np.min(klass_features, axis=0))/h
        )
    binwidths[binwidths==np.inf] = 0  # done with dividing, 
    bins[bins==0] = 1  # We always need at least one bin

    return bins


In [10]:
def fit_naive_bayes(
        features: np.ndarray,
        labels: np.ndarray,
        bincount: int
) -> Tuple[np.ndarray, np.ndarray]:
    """Fit the given features and labels for the naive bayes algorithm.

    Parameters
    ----------
    features : numpy.ndarray
        `X×D` dimensional array, 
        with `D` beeing the number of feature dimensions.
    labels : numpy.ndarray
        `X×1` dimensional array.
    bincount : int
        Number of bins for the histograms.

    Returns
    -------
    histograms : numpy.ndarray
        `C×D×L` dimensional array, with
        `C` as the number of unique classes,
        `D` as the number of feature dimensions and
        `L` as the number of bins.
    binning : numpy.ndarray
        `C×D×2` dimensional array describing the bin layout.
    """
    N: int = labels.size
    klasses: np.ndarray = np.unique(labels)
    C: int = klasses.size
    D: int = features.shape[1]
    L: int = bincount

    bincounts: np.ndarray
    if L == 0:
        bincounts = freedman_diaconis(features, labels)
        L = int(np.mean(bincounts))

    # let's start by predefining our arrays
    binning: np.ndarray
    histograms: np.ndarray
    binning = np.zeros((C, D, 2))
    histograms = np.zeros((C, D, L))
    for k, klass in enumerate(klasses):
        # filter our features by their class
        klass_features: np.ndarray = features[labels == klass]
        # cache the min and max features for further use
        min_feature: np.ndarray = np.min(klass_features, axis=0)
        max_feature: np.ndarray = np.max(klass_features, axis=0)

        # the first bin always starts at the lowest feature
        binning[k, :, 0] = min_feature  
        binning[k, :, 1] = (max_feature-min_feature)/L

        # since the calculated binwidth could be 0
        # we should change it to a better value for 
        # further calculations, I chose np.inf
        bins = binning[k, :, 1]
        bins[bins==0] = np.inf  # we cant have 0 width
        binning[k, :, 1] = bins

        histograms[k] = np.floor(
            (klass_features - binning[k, :, 0]) / binning[k, :, 1]
        ).T

    return histograms, binning


> Filter the training set to use only digits “3” and “9”

In [11]:
mask_39 = np.logical_or(data == 3, data == 9)

features_39 = data[mask_39]
labels_39 = target[mask_39]

X_train,\
    X_test,\
    y_train,\
    y_test = model_selection.train_test_split(
        features_39, labels_39, test_size=0.33, random_state=0
    )


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def predict_naive_bayes(
    test_features: np.ndarray,
    histograms: np.ndarray,
    binning: np.ndarray
) -> np.ndarray:
    """Predict the labels for the given test features using naive bayes.

    Parameters
    ----------
    test_features : np.ndarray
        `N×D` dimensional array, 
        with `D` beeing the number of feature dimensions.
    histograms : np.ndarray
        `C×D×L` dimensional array, with
        `C` as the number of unique classes,
        `D` as the number of feature dimensions and
        `L` as the number of bins.
    binning : np.ndarray
        `C×D×2` dimensional array describing the bin layout.


    Returns
    -------
    predicted_labels : np.ndarray
        `N×1` dimensional array, 
        with `N` beeing the number of feature instances.
    """
    N: int = test_features.shape[0]
    C: int = histograms.shape[0]
    D: int = test_features.shape[1]


    
    predicted_labels: np.ndarray
    feature_probabilities: np.ndarray
    for k in range(C):
        feature_probabilities = np.zeros(N)

        # Case 1: Our bins are defined of infinite size
        # this means that all our features are the same value
        maks_1 = binning[k , :, 1] == np.inf
            histograms[k, :, maks_1]



    
